# Movie Lens
Subset of the Movie Lens 25M dataset

# Setup, Vectorize and Load Data

In this tutorial, we'll demonstrate how to leverage a sample dataset stored in Azure Cosmos DB for MongoDB to ground OpenAI models. We'll do this taking advantage of Azure Cosmos DB for Mongo DB vCore's [vector similarity search](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) functionality. In the end, we'll create an interatice chat session with the GPT-3.5 completions model to answer questions about Azure services informed by our dataset. This process is known as Retrieval Augmented Generation, or RAG.

In [ ]:
! pip install openai
! pip install pymongo
! pip install python-dotenv
! pip install urlopen
! pip install azure-cosmos

In [ ]:
# Import the required libraries
import zipfile
import ast
import asyncio
from openai import AzureOpenAI
from dotenv import dotenv_values
import urllib
from tenacity import retry, stop_after_attempt, wait_random_exponential
from time import sleep
import time
import json
import uuid
from azure.cosmos.aio import CosmosClient
from azure.cosmos import exceptions, PartitionKey

# Load environment values and intantiate clients

In [ ]:
# Variables
# specify the name of the .env file name 
env_name = "../fabconf.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

cosmos_conn = config['cosmos_nosql_connection_string']
cosmos_key = config['cosmos_nosql_key']
cosmos_database = config['cosmos_database_name']
cosmos_collection = config['cosmos_collection_name']
cosmos_vector_property = config['cosmos_vector_property_name']
cosmos_cache = config['cosmos_cache_collection_name']

openai_endpoint = config['openai_endpoint']
openai_key = config['openai_key']
openai_api_version = config['openai_api_version']
openai_embeddings_deployment = config['openai_embeddings_deployment']
openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])
openai_completions_deployment = config['openai_completions_deployment']

# Create the Azure Cosmos DB for NoSQL client
cosmos_client = CosmosClient(url=cosmos_conn, credential=cosmos_key)
# Create the OpenAI client
openai_client = AzureOpenAI(azure_endpoint=openai_endpoint, api_key=openai_key, api_version=openai_api_version)

#  Create a database and containers with vector policies

This function takes a database object, a collection name, the name of the document property that will store vectors, and the number of vector dimensions used for the embeddings.

In [ ]:
db = await cosmos_client.create_database_if_not_exists(cosmos_database)

# Create the vector embedding policy to specify vector details
vector_embedding_policy = {
    "vectorEmbeddings": [ 
        { 
            "path":"/" + cosmos_vector_property,
            "dataType":"float32",
            "distanceFunction":"dotproduct",
            "dimensions":openai_embeddings_dimensions
        }, 
    ]
}

# Create the vector index policy to specify vector details
indexing_policy = {
    "vectorIndexes": [ 
        {
            "path": "/"+cosmos_vector_property, 
            "type": "quantizedFlat" 
        }
    ]
} 

# Create the data collection with vector index
try:
    container = await db.create_container_if_not_exists( id=cosmos_collection, 
                                                  partition_key=PartitionKey(path='/id'), 
                                                  vector_embedding_policy=vector_embedding_policy,
                                                  offer_throughput=1000) 
    print('Container with id \'{0}\' created'.format(id)) 

except exceptions.CosmosHttpResponseError: 
        raise 

# Create the cache collection with vector index
try:
    cache_container = await db.create_container_if_not_exists( id=cosmos_cache, 
                                                  partition_key=PartitionKey(path='/id'), 
                                                  indexing_policy=indexing_policy,
                                                  vector_embedding_policy=vector_embedding_policy,
                                                  offer_throughput=1000) 
    print('Container with id \'{0}\' created'.format(id)) 

except exceptions.CosmosHttpResponseError: 
        raise 


# Generate embeddings from Azure OpenAI

We'll create a a helper function to generate embeddings from passed in text using Azure OpenAI. We'll also add a retry to handle any throttling due to quota limits.


In [ ]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(20))
def generate_embeddings(text):
    
    response = openai_client.embeddings.create(
        input=text,
        model=openai_embeddings_deployment,
        dimensions=openai_embeddings_dimensions
    )
    
    embeddings = response.model_dump()
    return embeddings['data'][0]['embedding']

# Load the data from the JSON file

In [ ]:
# Unzip the data file
with zipfile.ZipFile("../Data/MovieLens-4489-256D.zip", 'r') as zip_ref:
    zip_ref.extractall("../Data")
zip_ref.close()

In [ ]:
# Load the data file
data =[]
with open('../Data/MovieLens-4489-256D.json', 'r') as d:
    data = json.load(d)

In [ ]:
# Peek at the first document
data[0]

In [ ]:
# View the number of documents in the data (4489)
len(data) 

# Store data in Azure Cosmos DB. 
Upsert data into Azure Cosmos DB for NoSQL. Optionally, vectorize properties of the document (this has been done in the sample data)

In [ ]:
async def insert_data():
        #stream = urllib.request.urlopen(storage_file_url)
        counter = 0
        list_to_upsert = []
        await cosmos_client.__aenter__()
        for object in data:

                #The following code to create vector embeddings for the data is commented out as the sample data is already vectorized.
                #vectorArray = generate_embeddings("Title:" + data[i]['original_title'] + ", Tagline:" + data[i]['tagline'] + ", Overview:" + data[i]['overview'])
                #object[cosmos_vector_property] = vectorArray
                await container.upsert_item(body=object)

                # print progress every 100 upserts. 
                counter += 1
                if counter % 100 == 0:
                        print("Inserted {} documents into collection.".format(counter))
        print ("Upsert completed!")

Now we insert the data.

In [ ]:
# Insert the data asynchronously
await insert_data()

Now you're ready to start building your Chatbot!